In [1]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


In [2]:
import sys
import os
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

from read_data.read_data import read_urls_questions, get_questions, get_url_content_tuples, get_relevant_docs
from elasticsearch import Elasticsearch
from retrieval.sparse_retrieval.bm25 import set_index, get_result_tuples

In [3]:
es_client = Elasticsearch("http://localhost:9200")

In [4]:
es_client.info().body

{'name': 'c913d73a452f',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': '2OYk7KTMQx6zkjPCEjNf2Q',
 'version': {'number': '8.7.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '09520b59b6bc1057340b55750186466ea715e30e',
  'build_date': '2023-03-27T16:31:09.816451435Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [5]:
collected, questions =  read_urls_questions('data/all_references/passages_100words.csv',
                                            'data/question_answer/all_ref_questions.csv') # read collected urls and questions + remove unsuccessful collection

In [6]:
print(len(collected))
print(len(questions))

13711
451


In [7]:
question_list = get_questions(questions)
document_list = get_url_content_tuples(collected)
mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [8]:
es_client = set_index(es_client=es_client, collected=collected, mappings=mappings) # 7secs

In [25]:
results = get_result_tuples(es_client=es_client, questions=question_list, n=10) # 1:45 min

In [60]:
list(results.keys())[40]

'\n \n3. Welke windturbines zijn er in Amsterdam binnen de kaders van het activiteitenbesluit en \nde activiteitenregeling gebouwd en welke niet? \n \n'

In [61]:
questions.iloc[40]

Unnamed: 0.1                                                   40
Unnamed: 0                                                     55
Year                                                         2021
Month                                                           6
Question        \n \n3. Welke windturbines zijn er in Amsterda...
Answer            \nOp 1 januari 2008 hebben twaalf brancheger...
Document        https://amsterdam.raadsinformatie.nl/document/...
URLs            ['https://www.infomil.nl/onderwerpen/integrale...
Cleaned_URLs               ['https://windstats.nl/statistieken/']
Name: 40, dtype: object

In [63]:
#results[list(results.keys())[40]][1]

In [29]:
from irmetrics.topk import recall
predictions = []
for question in results.keys():
    urls = []
    for result in results[question][1]:
        urls.append(result['url'])
    predictions.append(list(set(urls)))


relevant_docs = get_relevant_docs(questions)

true = []
for question in relevant_docs.keys():
    true.append(list(set(relevant_docs[question])))

In [30]:
import numpy as np
from irmetrics.topk import rr

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = rr(true_values, predictions[i], k =10)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_mrr = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_mrr)


Average Mean Reciprocal Rank: 0.0


In [31]:
from irmetrics.topk import recall

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = recall(true_values, predictions[i], k=10)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_recall = np.mean(mrr_values)

print("Average Recall:", average_recall)

Average Recall: 0.0


In [64]:
import pickle
with open ('data/results/bm25_results_all_ref_passages.pickle', 'wb') as f:
    pickle.dump(results, f)

In [70]:
results[list(results.keys())[0]][1][0]['text']

'Kamerbrief over de gevolgen van de uitspraak van de Raad van State over de milieubeoordeling voor windturbinenormen | Kamerstuk | Rijksoverheid.nl Ga direct naar inhoud Rijksoverheid U bevindt zich hier : Home Documenten Kamerbrief over de gevolgen van de uitspraak van de Raad van State over de milieubeoordeling voor windturbinenormen Zoeken binnen Rijksoverheid.nl Zoek Kamerbrief over de gevolgen van de uitspraak van de Raad van State over de milieubeoordeling voor windturbinenormen Kamerstuk | 06-07-2021Staatssecretaris Yeşilgöz informeert de Eerste en Tweede Kamer over de gevolgen van de uitspraak van de Afdeling Bestuursrechtspraak van de Raad van State van 30 juni 2021'